### 1.1 Listar Procesos Activos:

    Crea un programa que pida una lista de nombres de procesos, y muestre por pantalla si dichos procesos están activos indicando el nombre, PID, y uso de memoria, o indicando que no se están ejecutando en caso contrario.
    Filtra los procesos para que solo se muestren aquellos cuyo nombre contiene una palabra clave especificada por el usuario (por ejemplo, "chrome" o "explorer").
    Maneja las posibles excepciones, como acceso denegado a ciertos procesos.

In [ ]:
import psutil       # Import necesario para ejercicio 1

In [ ]:
def listar_procesos(clave):
    procesos = []
    for proc in psutil.process_iter(['pid', 'name', 'memory_info']):
        try:
            if clave.lower() in proc.info['name'].lower():
                procesos.append(proc.info)
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    
    return procesos


if __name__ == "__main__":
    clave = input("Ingrese palabra clave para filtrar procesos: ")
    procesos = listar_procesos(clave)
    
    if procesos:
        for p in procesos:
            print(f"Nombre: {p['name']}, PID: {p['pid']}, Memoria: {p['memory_info'].rss / (1024 * 1024):.2f} MB")
    else:
        print("No se encontraron procesos activos con la palabra clave.")


### 1.2 Finalizar Procesos Específicos:

    Modifica el programa anterior para permitir al usuario seleccionar un proceso por su nombre y finalizarlo.
    Muestra un mensaje de confirmación si el proceso se finaliza correctamente o un mensaje de error si no se puede finalizar.

In [ ]:
def finalizar_proceso(nombre_proceso):
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if nombre_proceso.lower() == proc.info['name'].lower():
                proc.terminate()
                proc.wait()  # Espera a que termine
                print(f"Proceso {nombre_proceso} (PID: {proc.info['pid']}) finalizado.")
                return
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    print(f"No se pudo finalizar el proceso {nombre_proceso}.")


if __name__ == "__main__":
    nombre = input("Ingrese el nombre del proceso a finalizar: ")
    finalizar_proceso(nombre)


### 2.1 Envío de Mensajes entre Procesos:

    Crea un programa que utilice os.fork() para crear un proceso hijo.
    El proceso padre debe enviar un mensaje al hijo a través de un pipe, y el proceso hijo debe responder con una versión modificada del mensaje (en mayúsculas).
    El padre debe mostrar el mensaje modificado recibido del hijo.

In [ ]:
import os       # Import necesario para ejercicio 2

In [ ]:
# (Ejecutado y probado en colab, aquí no funciona por ser Windows)

def proceso_padre():
    r, w = os.pipe()
    pid = os.fork()
    
    if pid > 0:  # Proceso padre
        os.close(r)
        mensaje = "Hola, hijo"
        os.write(w, mensaje.encode())
        os.close(w)
    else:  # Proceso hijo
        os.close(w)
        r = os.fdopen(r)
        mensaje = r.read()
        print(f"Proceso hijo recibió: {mensaje}")
        print(f"Respuesta modificada: {mensaje.upper()}")
        r.close()


if __name__ == "__main__":
    proceso_padre()


### 2.2 Intercambio de Archivos:

    Extiende el programa anterior para que el proceso padre envíe el contenido de un archivo de texto al hijo a través del pipe.
    El proceso hijo debe contar el número de líneas y palabras del archivo y enviar esa información de vuelta al padre.

In [ ]:
# ¿Como puedo probar si está bien? Preguntar

def proceso_padre_archivo(ruta_archivo):
    r, w = os.pipe()
    pid = os.fork()

    if pid > 0:  # Proceso padre
        os.close(r)
        with open(ruta_archivo, 'r') as file:
            contenido = file.read()
        os.write(w, contenido.encode())
        os.close(w)
    else:  # Proceso hijo
        os.close(w)
        r = os.fdopen(r)
        contenido = r.read()
        lineas = contenido.count('\n')
        palabras = len(contenido.split())
        print(f"Proceso hijo: {lineas} líneas, {palabras} palabras")
        r.close()

if __name__ == "__main__":
    proceso_padre_archivo("archivo.txt")


### 3. Comparación de Ejecución:
        Crea un programa que ejecute el Bloc de notas (Notepad.exe) de manera síncrona y asíncrona.
        Mide y muestra el tiempo de ejecución en ambos casos para ilustrar la diferencia entre la ejecución bloqueante y no bloqueante.
        El programa debe permitir al usuario elegir entre ejecución síncrona o asíncrona.

In [ ]:
# imports de ejercicio 3

import subprocess
import time
import asyncio

In [ ]:
def ejecutar_sincronamente():       
    start_time = time.time()
    print("Ejecución del NotePad síncrona")
    subprocess.run(["notepad.exe"])         # Hasta que no cierre el block de notas no paso de esta línea
    print(f"Tiempo de ejecución síncrona: {time.time() - start_time:.2f} segundos")     

async def ejecutar_asincronamente():
    start_time = time.time()
    print("Ejecución del NotePad asíncrona")
    try:
        await asyncio.create_subprocess_exec('notepad.exe')          # En cuanto se lea la línea pasa a la siguiente (entendiéndose que si va bien la siguiente es el print, no el except)
    except subprocess.CalledProcessError as e:
        print(e.output)    
    print(f"Tiempo de ejecución asíncrona: {time.time() - start_time:.2f} segundos")

if __name__ == "__main__":

    ejecutar_sincronamente()

    await ejecutar_asincronamente()


### 4: Transferencia de Datos y Manipulación del Portapapeles (30-45 minutos)

    Automatización con FTP y Portapapeles:
        Escribe un programa que se conecte a un servidor FTP público utilizando subprocess.Popen(), realice una autenticación y descargue un archivo.
        El contenido del archivo descargado debe ser copiado al portapapeles.
        Implementa la verificación periódica del portapapeles para detectar si el contenido ha cambiado. Si es así, muestra un mensaje en la consola.

In [ ]:
# imports de ejercicio 4

import subprocess

In [ ]:
# Parte a

p1 = subprocess.Popen('ftp', shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

comandos = [b"verbose\n",             
            b"open test.rebex.net\n", 
            b"demo\n",                
            b"password\n",            
            b"ls\n",                 
            b"get readme.txt\n"]   

for cmd in comandos:
    p1.stdin.write(cmd) 

respuesta = p1.communicate(timeout=5)[0]

print(respuesta.decode("cp850", "ignore"))


Modo detallado Activo .
Conectado a test.rebex.net.
220-Welcome to test.rebex.net!
    See https://test.rebex.net/ for more information and terms of use.
220 If you don't have an account, log in as 'anonymous' or 'ftp'.
200 Enabled UTF-8 encoding.
Usuario (test.rebex.net:(none)): 331 Anonymous login OK, send your complete email address as your password.
Contraseña: 
230 User 'demo' logged in.
200 'PORT' OK.
150 Opening 'BINARY' data connection.
pub
readme.txt
226 Transfer complete.
ftp: 20 bytes recibidos en 0.00segundos 20000.00a KB/s.
200 'PORT' OK.
150 Opening 'BINARY' data connection.
226 Transfer complete.
ftp: 379 bytes recibidos en 0.00segundos 379000.00a KB/s.
221 Closing session.



In [ ]:
# Parte b

def copiar_porta(texto):
    p = subprocess.run('clip', input=texto.strip(), text=True, shell=True)
    return p.returncode     # 0 bien, else mal

with open("readme.txt", 'r') as fichero:  # Usar with para manejar el archivo
    x = fichero.read()
    print(copiar_porta(x) )     # Para comprobar que funciona


# Parte 

import pyperclip
import time

def monitor_portapapeles():
    ultimo_contenido = pyperclip.paste()  # Obtener el contenido inicial del portapapeles
    print("Monitoreando portapapeles...")

    while ultimo_contenido!="Salir".casefold():     # casefold() hace que me de igual si tiene mayúsculas o minúsculas
        time.sleep(1)  # Esperar 1 segundo entre verificaciones
        contenido_actual = pyperclip.paste()

        if contenido_actual != ultimo_contenido:
            print(f"¡El portapapeles ha cambiado!\nNuevo contenido: {contenido_actual}.\nPara salir copie al portapapeles \"Salir\"")
            ultimo_contenido = contenido_actual  # Actualizar el último contenido

if __name__ == "__main__":
    monitor_portapapeles()


0
Monitoreando portapapeles...
¡El portapapeles ha cambiado!
Nuevo contenido: salir.
Para salir copie al portapapeles "Salir"
